In [47]:
from threading import Thread, Condition, Lock
from time import sleep, time
from IPython.display import clear_output
import random
import copy
import sys

def chunks(arr, n):
    for i in range(0, len(arr), n):
        yield arr[i:i + n]


i_min_max(arr, flag):
    ma = float(flag)
    i_ma = 0
    for i, v in enumerate(arr):
        if v > ma:
            ma = v
            i_ma = i
    return i_ma, ma


def i_max(arr):
    return i_min_max(arr, '-inf')

def i_min(arr):
    return i_min_max(arr, 'inf')

In [2]:
class Environment(object):
    #multiplying by this wil be equivalen to (%): [0, 25, 45, 60, 70, 75, 78, 80, 80]
    _concurrent_loss= [1, 0.75, 0.55, 0.4, 0.3, 0.25, 0.22, 0.2, 0.2]

    _audience = [
        [2,   1.3,    1],
        [1.3, 0.9, 0.75],
        [1,  0.75, 0.47]
    ]

    _hour_coef = [
        [ -1, 0.55, 0.45,  -1],
        [ -1,  0.7, 0.75,  -1],
        [ -1,  0.8,  0.5,  -1],
        [0.4,    1,    1, 0.4]
    ]

    _categories = ['A', 'B', 'C']
    _days = ['V', 'S', 'D', 'L']
    _hours = [12, 16, 18, 20]

    _days_hours = {
        0: [3],
        1: [0, 1, 2, 3],
        2: [0, 1, 2, 3],
        3: [3]
    }



class Match(object):
    def __init__(self, idx_homeCat, idx_awayCat, idx_day, idx_hour):
        self._hCat = idx_homeCat
        self._aCat = idx_awayCat
        self._d = idx_day
        self._h = idx_hour

    def get_raw_time(self):
        return [self._d, self._h]

    def get_away_category(self):
        return Environment._categories[self._aCat]

    def get_home_category(self):
        return Environment._categories[self._hCat]

    def get_base(self):
        return Environment._audience[self._hCat][self._aCat]

    def get_hour_coef(self):
        return Environment._hour_coef[self._h][self._d]

    def get_score(self):
        return self.get_base() * self.get_hour_coef()


class GenotypeTimetable(object):
    def __init__(self, num_matches=20, chain=None):
        '''
        Genotype representation, list of lists: [[1, 2]] ([[D, H], [D, H]])
        where in pos 0 is the index of day, and in pos 1 the index of hour
        '''
        if chain != None:
            self._chain = chain
        else:
            self._chain = []
            for i in range(num_matches):
                days = list(Environment._days_hours.keys())
                idx_d = random.choice(days)
                idx_h = random.choice(Environment._days_hours[idx_d])

                self._chain.append([idx_d, idx_h])


    def get_applciable_chain(self):
        l = []
        for (day, hour) in self._chain:
            l.append([Environment._days[day], Environment._hours[hour]])

        return l


    def mutate(self):
        chrom = random.choice(range(len(self._chain)))
        days = list(Environment._days_hours.keys())
        idx_d = random.choice(days)
        idx_h = random.choice(Environment._days_hours[idx_d])
        self._chain[chrom][0] = idx_d
        self._chain[chrom][1] = idx_h


    def cross(self, genotype):
        g1 = copy.deepcopy(self._chain)
        g2 = copy.deepcopy(genotype._chain)

        sl = random.choice(range(len(g1)))
        first = random.choice([0, 1])

        g3 = []
        if first == 0:
            g3 = g1[:sl] + g2[sl:]
        else:
            g3 = g2[:sl] + g1[sl:]

        return GenotypeTimetable(chain=g3)
        

    def count(self, chrom):
        return self._chain.count(chrom)

    def __str__(self):
        return '{}'.format(self.get_applciable_chain())

    def __repr__(self):
        return '{}'.format(self.get_applciable_chain())

    def __getitem__(self, idx):
        return self._chain[idx]

    def __len__(self):
        return len(self._chain)



class FenotypeTimetable(object):
    def __init__(self, matchCombinations):
        '''
        :param matchCombinations: List of lists of combinations of matches (e.g: [[A, A], [A, B]])
        '''
        self._m_comb = []
        for (cat1, cat2) in matchCombinations:
            self._m_comb.append([Environment._categories.index(cat1), Environment._categories.index(cat2)])


    def set_genotype(self, genotype):
        self._gen = genotype
        self._matches = []
        for i, (cat1, cat2) in enumerate(self._m_comb):
            (day, hour) = genotype[i]
            self._matches.append(Match(cat1, cat2, day, hour))


    def get_score(self):
        acum = 0
        for match in self._matches:
            raw_time = match.get_raw_time()
            coincidences = self._gen.count(raw_time) - 1
            
            coin_coef = 0.2 #if not in list give an 80% loss
            if coincidences < len(Environment._concurrent_loss):
                coin_coef = Environment._concurrent_loss[coincidences]

            sc = match.get_score()
            acum += sc * coin_coef

        return acum




gen1 = GenotypeTimetable(5)
gen2 = GenotypeTimetable(5)
print('Gen 1:', gen1)
print('Gen 2:', gen2)

print()
gen3 = gen1.cross(gen2)
print('Gen 1 + 2 (3)   :', gen3)
gen3.mutate()
print('Gen 3 (mutation):', gen3)
print()
fen = FenotypeTimetable([['A', 'A'], ['B', 'A'], ['A', 'C'], ['A', 'B'], ['C', 'C']])
fen.set_genotype(gen3)
match1 = fen._matches[0]
print('Match 1 info: ', match1.get_home_category(), match1.get_away_category(), match1.get_base(), match1.get_hour_coef(), match1.get_score())
print('Match scores: ', [i.get_score() for i in fen._matches])
print('Total score (fitness): ', fen.get_score())

Gen 1: [['S', 18], ['S', 12], ['V', 20], ['V', 20], ['V', 20]]
Gen 2: [['D', 20], ['L', 20], ['S', 12], ['L', 20], ['L', 20]]

Gen 1 + 2 (3)   : [['S', 18], ['S', 12], ['V', 20], ['L', 20], ['L', 20]]
Gen 3 (mutation): [['V', 20], ['S', 12], ['V', 20], ['L', 20], ['L', 20]]

Match 1 info:  A A 2 0.4 0.8
Match scores:  [0.8, 0.7150000000000001, 0.4, 0.52, 0.188]
Total score (fitness):  2.1460000000000004


In [51]:
class Genetic(object):
    def __init__(self, matches, living_things=8, iterations=50, reproductions=1, mutations=1, alpha=10, n_threads=4, selection='tournament', tournament_percent=0.3):
        self.MATCHES = matches

        self.__living_things = living_things
        self.__iterations = iterations
        self.__reproductions = reproductions
        self.__mutations = mutations
        self.__n_threads = n_threads
        self.__alpha = alpha
        self.__selection = selection
        self.__tournament_percent = tournament_percent

        self.fitness_list = []
        self.genotypes = []

        self.best_genotype = None
        self.best_fenotype = None
        self.best_score = -1


    def init(self):
        #init population
        l = len(self.MATCHES)
        for i in range(self.__living_things):
            gen = GenotypeTimetable(l) #initialize gen

            self.genotypes.append(gen)
            self.fitness_list.append(0)


    @staticmethod
    def evaluation_thread(gen, matches, fitness_list, index):
        fen = FenotypeTimetable(matches)
        fen.set_genotype(gen)
        fitness_list[index] = fen.get_score()


    def evaluation(self):
        i = -1

        #Divide in threads
        for genotype_chunk in chunks(self.genotypes, self.__n_threads):
            running_threads = []
            for gen in genotype_chunk:
                i += 1
                T = Thread(
                        target=Genetic.evaluation_thread,
                        args=(gen, self.MATCHES, self.fitness_list, i))

                running_threads.append(T)
                T.start()


            for thread in running_threads:
                thread.join()


    def selection_cross(self):
        if self.__selection == 'tournament':
            fitness_samples = int(len(self.fitness_list) * self.__tournament_percent)
            for i in range(self.__reproductions):
                i_fitnesses = []
                for n in range(2): #2 fathers, 1 per tournament
                    fitnesses = []
                    for fit in range(fitness_samples): #tournament
                        fitnesses.append(random.choice(self.fitness_list))

                    i_fitnesses.append(i_max(fitnesses)[0]) #select the best in the tournament

                #reproduce the fathers selected in the tournaments
                gen1 = self.genotypes[i_fitnesses[0]]
                gen2 = self.genotypes[i_fitnesses[1]]
                self.genotypes.append(gen1.cross(gen2))
                self.fitness_list.append(0)

        else:
            wl = []
            for i, fit in enumerate(self.fitness_list):
                wl += [i] * int(fit)
            
            for i in range(self.__reproductions):
                #reproduce gen
                gen = self.genotypes[random.choice(wl)].cross(self.genotypes[random.choice(wl)])

                self.genotypes.append(gen)
                self.fitness_list.append(0)


    def mutation(self):
        for i in range(self.__mutations):
            for gen in self.genotypes:
                gen.mutate()


    def reselection(self):
        if self.__selection == 'tournament':
            fitness_samples = int(len(self.fitness_list) * self.__tournament_percent)
            for i in range(self.__reproductions):
                fitnesses = []
                for fit in range(fitness_samples): #tournament
                    fitnesses.append(random.choice(self.fitness_list))

                idx = (i_min(fitnesses)[0]) #select the worst in the tournament

                del self.genotypes[idx]
                del self.fitness_list[idx]

        else:
            for i in range(self.__reproductions):
                ma = max(self.fitness_list) + 1
                wl = []
                for i, fit in enumerate(self.fitness_list):
                    wl += [i] * int(ma - fit)

                idx = random.choice(wl)

                del self.genotypes[idx]
                del self.fitness_list[idx]


    def select_solution(self):
        for i, fit in enumerate(self.fitness_list):
            if fit > self.best_score:
                self.best_score = fit
                self.best_genotype = self.genotypes[i]


    def optimize(self):
        self.init()

        self.evaluation()

        #loop
        progress_size = 50
        print('[>', end='')
        _s = 0
        _p = 0
        step = self.__iterations / 50
        for i in range(self.__iterations):
            self.selection_cross()
            self.mutation()
            self.evaluation()
            self.reselection()
            self.select_solution()

            if _s >= step:
                _p += 1
                _s = 0
                clear_output(wait=True)
                print('[{}>{}] {:.2f}%'.format(''.join(['=']*_p), ''.join([' ']*(progress_size-_p)), _p/progress_size*100),  end='')

            _s += 1

        _p += 1
        clear_output(wait=True)
        print('[{}>{}] {:.2f}%'.format(''.join(['=']*_p), ''.join([' ']*(progress_size-_p)), _p/progress_size*100),  end='')
        print(' DONE!')


genetic = Genetic([['B', 'A'], ['B', 'A'], ['C', 'C'], ['B', 'A'], ['C', 'C'], ['B', 'C'], ['B', 'B'], ['B', 'B'], ['B', 'C'], ['A', 'B']], iterations=1000, living_things=50, mutations=1, reproductions=5, alpha=1, n_threads=8, selection='tournament', tournament_percent=0.3)

t = time()
genetic.optimize()
print('Best score: ', genetic.best_score)
print('Best genotype: ', genetic.best_genotype)
print('Elapsed: ', (time() - t) / 60, 'mins')

[==================================================>] 100.00% DONE!
Best score:  6.516000000000001
Best genotype:  [['D', 18], ['S', 12], ['D', 18], ['S', 20], ['V', 20], ['V', 20], ['L', 20], ['L', 20], ['S', 16], ['V', 20]]
Elapsed:  0.30164308150609337 mins
